In [10]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import operator
import tensorflow as tf
import random
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model


import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from keras.models import load_model
import numpy as np 
import pandas as pd 
import os
import glob
import cv2
import random
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import regularizers
from keras.callbacks import CSVLogger
#from livelossplot import PlotLossesKeras
import os
import numpy as np
#from imgaug import augmenters as iaa
#import cv2
from keras.layers.normalization import BatchNormalization
#import seaborn as sns
import pandas as pd 
from keras import initializers
from keras import optimizers
import keras.backend as K
import tensorflow as tf
from keras.models import Model

In [11]:
TRAIN_PATH = r'D:/dataset/Train'
VAL_PATH = r'D:/dataset/Test'
BATCH_SIZE=16
r=4
c=4
#CATEGORIES = ['Air_trapping', 'Aortic_elongation','COPD_Signs','Calcified_granuloma','Callus_rib_fracture','Hiatal_hernia','Kyphosis','Laminar_atelectasis','Normal','Pleural_effusion','Scoliosis','Vascular_hilar_enlargement']

In [12]:
train_datagen = ImageDataGenerator(rescale=1.0/255.0)
train_batches = train_datagen.flow_from_directory(TRAIN_PATH,
                                                  class_mode='categorical', 
                                                  batch_size=BATCH_SIZE, 
                                                  target_size=(256, 256),
                                                  shuffle=True,
                                                  seed=42
                                                  )

Found 34387 images belonging to 12 classes.


In [13]:
val_datagen = ImageDataGenerator(rescale=1.0/255.0)
val_batches = val_datagen.flow_from_directory(VAL_PATH,
                                                class_mode='categorical', 
                                          
                                                batch_size=BATCH_SIZE, 
                                                target_size=(256, 256),
                                                shuffle=True,
                                                seed=42
                                                )

Found 8597 images belonging to 12 classes.


In [14]:
from keras.applications.vgg16 import VGG16

In [15]:
from keras.optimizers import Adam

In [16]:
def vgg():
    base_model = VGG16(weights='imagenet',include_top=False,pooling='avg',input_shape=(256,256,3))
    predictions=Dense(12,activation='softmax',trainable=True)(base_model.output)
    for layer in base_model.layers:
        layer.trainable=True
    model=Model(inputs=[base_model.input], outputs=[predictions])
    
    optim = tf.keras.optimizers.Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-4)
    loss_func = 'mae'
    
    model.compile(optimizer=optim,loss=loss_func,metrics=['accuracy'])
    return model

model=None
model = vgg()
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)      

In [17]:
#import keras
import tensorflow.keras as keras
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
callbacks = [
    tf.keras.callbacks.ModelCheckpoint('adam_baseline_vgg.h5', monitor='val_accuracy', save_best_only=True, mode='max'),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, verbose=1, patience=5, mode='max')]


In [18]:
history  = model.fit(train_batches, 
         steps_per_epoch=train_batches.n//train_batches.batch_size,
         validation_data=val_batches, 
         validation_steps=val_batches.n//val_batches.batch_size, 
         epochs=20, 
         verbose=1,
         callbacks = callbacks)

Epoch 1/20
2149/2149 [==============================] - 1333s 620ms/step - loss: 0.0351 - accuracy: 0.7907 - val_loss: 0.0344 - val_accuracy: 0.7939
Epoch 2/20
2149/2149 [==============================] - 501s 233ms/step - loss: 0.0343 - accuracy: 0.7940 - val_loss: 0.0344 - val_accuracy: 0.7939
Epoch 3/20
2149/2149 [==============================] - 513s 239ms/step - loss: 0.0343 - accuracy: 0.7941 - val_loss: 0.0343 - val_accuracy: 0.7940
Epoch 4/20
2149/2149 [==============================] - 515s 239ms/step - loss: 0.0343 - accuracy: 0.7940 - val_loss: 0.0344 - val_accuracy: 0.7939
Epoch 5/20
2149/2149 [==============================] - 513s 239ms/step - loss: 0.0343 - accuracy: 0.7940 - val_loss: 0.0344 - val_accuracy: 0.7939
Epoch 6/20
2149/2149 [==============================] - 509s 237ms/step - loss: 0.0343 - accuracy: 0.7940 - val_loss: 0.0344 - val_accuracy: 0.7938
Epoch 7/20
2149/2149 [==============================] - 512s 238ms/step - loss: 0.0343 - accuracy: 0.7939 - val

In [19]:
model.save('baseline.h5')

In [21]:
test_generator = ImageDataGenerator()
test_data_generator = test_generator.flow_from_directory(
    'D:/dataset/Test', # Put your path here
     target_size=(256, 256),
    batch_size=16,
    shuffle=False)
test_steps_per_epoch = np.math.ceil(test_data_generator.samples / test_data_generator.batch_size)

predictions = model.predict_generator(test_data_generator, steps=test_steps_per_epoch)
# Get most likely class
predicted_classes = np.argmax(predictions, axis=1)

Found 8597 images belonging to 12 classes.
Instructions for updating:
Please use Model.predict, which supports generators.


In [22]:
true_classes = test_data_generator.classes
class_labels = list(test_data_generator.class_indices.keys())   

In [24]:
import sklearn.metrics as metrics

In [25]:
report = metrics.classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)    

                            precision    recall  f1-score   support

              Air_trapping       0.00      0.00      0.00       136
         Aortic_elongation       0.00      0.00      0.00       147
                COPD_Signs       0.00      0.00      0.00       679
       Calcified_granuloma       0.00      0.00      0.00        68
       Callus_rib_fracture       0.00      0.00      0.00        66
             Hiatal_hernia       0.00      0.00      0.00        56
                  Kyphosis       0.00      0.00      0.00        35
       Laminar_atelectasis       0.00      0.00      0.00        95
                    Normal       0.79      1.00      0.89      6825
          Pleural_effusion       0.00      0.00      0.00        79
                 Scoliosis       0.00      0.00      0.00       352
Vascular_hilar_enlargement       0.00      0.00      0.00        59

                  accuracy                           0.79      8597
                 macro avg       0.07      0.0

C:\Users\Havoc\anaconda3\envs\padchest\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Havoc\anaconda3\envs\padchest\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Havoc\anaconda3\envs\padchest\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
